In [1]:
import h5py
import pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import shap
import lightgbm as lgb 
import optuna
from optuna.samplers import TPESampler
from optuna.integration import LightGBMPruningCallback
from optuna.pruners import MedianPruner
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import mean_absolute_error 
from sklearn.feature_selection import SelectKBest, f_regression, chi2, f_classif, mutual_info_regression
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from itertools import cycle, islice

In [2]:
# read data
def load_data(name):
    with h5py.File(f'{name}.h5', 'r') as f:
        filename = name.split('/')[-1]
        return pandas.DataFrame(f[filename][:], dtype=np.float64)

train = load_data('train')
test  = load_data('test')

print (f'Shape of training data set: {train.shape}')
print (f'Shape of test data set: {test.shape}')

all_variables = ['actualInteractionsPerCrossing', 'averageInteractionsPerCrossing', 'correctedActualMu', 'correctedAverageMu', 'correctedScaledActualMu', 'correctedScaledAverageMu', 'NvtxReco', 'p_nTracks', 'p_pt_track', 'p_eta', 'p_phi', 'p_charge', 'p_qOverP', 'p_z0', 'p_d0', 'p_sigmad0', 'p_d0Sig', 'p_EptRatio', 'p_dPOverP', 'p_z0theta', 'p_etaCluster', 'p_phiCluster', 'p_eCluster', 'p_rawEtaCluster', 'p_rawPhiCluster', 'p_rawECluster', 'p_eClusterLr0', 'p_eClusterLr1', 'p_eClusterLr2', 'p_eClusterLr3', 'p_etaClusterLr1', 'p_etaClusterLr2', 'p_phiClusterLr2', 'p_eAccCluster', 'p_f0Cluster', 'p_etaCalo', 'p_phiCalo', 'p_eTileGap3Cluster', 'p_cellIndexCluster', 'p_phiModCalo', 'p_etaModCalo', 'p_dPhiTH3', 'p_R12', 'p_fTG3', 'p_weta2', 'p_Reta', 'p_Rphi', 'p_Eratio', 'p_f1', 'p_f3', 'p_Rhad', 'p_Rhad1', 'p_deltaEta1', 'p_deltaPhiRescaled2', 'p_TRTPID', 'p_TRTTrackOccupancy', 'p_numberOfInnermostPixelHits', 'p_numberOfPixelHits', 'p_numberOfSCTHits', 'p_numberOfTRTHits', 'p_numberOfTRTXenonHits', 'p_chi2', 'p_ndof', 'p_SharedMuonTrack', 'p_E7x7_Lr2', 'p_E7x7_Lr3', 'p_E_Lr0_HiG', 'p_E_Lr0_LowG', 'p_E_Lr0_MedG', 'p_E_Lr1_HiG', 'p_E_Lr1_LowG', 'p_E_Lr1_MedG', 'p_E_Lr2_HiG', 'p_E_Lr2_LowG', 'p_E_Lr2_MedG', 'p_E_Lr3_HiG', 'p_E_Lr3_LowG', 'p_E_Lr3_MedG', 'p_ambiguityType', 'p_asy1', 'p_author', 'p_barys1', 'p_core57cellsEnergyCorrection', 'p_deltaEta0', 'p_deltaEta2', 'p_deltaEta3', 'p_deltaPhi0', 'p_deltaPhi1', 'p_deltaPhi2', 'p_deltaPhi3', 'p_deltaPhiFromLastMeasurement', 'p_deltaPhiRescaled0', 'p_deltaPhiRescaled1', 'p_deltaPhiRescaled3', 'p_e1152', 'p_e132', 'p_e235', 'p_e255', 'p_e2ts1', 'p_ecore', 'p_emins1', 'p_etconeCorrBitset', 'p_ethad', 'p_ethad1', 'p_f1core', 'p_f3core', 'p_maxEcell_energy', 'p_maxEcell_gain', 'p_maxEcell_time', 'p_maxEcell_x', 'p_maxEcell_y', 'p_maxEcell_z', 'p_nCells_Lr0_HiG', 'p_nCells_Lr0_LowG', 'p_nCells_Lr0_MedG', 'p_nCells_Lr1_HiG', 'p_nCells_Lr1_LowG', 'p_nCells_Lr1_MedG', 'p_nCells_Lr2_HiG', 'p_nCells_Lr2_LowG', 'p_nCells_Lr2_MedG', 'p_nCells_Lr3_HiG', 'p_nCells_Lr3_LowG', 'p_nCells_Lr3_MedG', 'p_pos', 'p_pos7', 'p_poscs1', 'p_poscs2', 'p_ptconeCorrBitset', 'p_ptconecoreTrackPtrCorrection', 'p_r33over37allcalo', 'p_topoetconeCorrBitset', 'p_topoetconecoreConeEnergyCorrection', 'p_topoetconecoreConeSCEnergyCorrection', 'p_weta1', 'p_widths1', 'p_widths2', 'p_wtots1', 'p_e233', 'p_e237', 'p_e277', 'p_e2tsts1', 'p_ehad1', 'p_emaxs1', 'p_fracs1', 'p_DeltaE', 'p_E3x5_Lr0', 'p_E3x5_Lr1', 'p_E3x5_Lr2', 'p_E3x5_Lr3', 'p_E5x7_Lr0', 'p_E5x7_Lr1', 'p_E5x7_Lr2', 'p_E5x7_Lr3', 'p_E7x11_Lr0', 'p_E7x11_Lr1', 'p_E7x11_Lr2', 'p_E7x11_Lr3', 'p_E7x7_Lr0', 'p_E7x7_Lr1' ]

all = train[all_variables]

electron_truth = train['Truth']
y = train['p_truth_E']
y = y[electron_truth==1]
X = train[all_variables]
X = X[electron_truth==1]
y = y.to_numpy(dtype='float32')

# print(y)
# prepocessing

norm = preprocessing.MinMaxScaler()
sc_X = preprocessing.StandardScaler()
sc_y = preprocessing.StandardScaler()
X = sc_X.fit_transform(X)
y = np.reshape(y, (-1,1))
y = sc_y.fit_transform(y)
y = y.ravel()

print(y)

X = pd.DataFrame(X, columns=all_variables)
# y = pd.DataFrame(y, columns=['p_truth_E'])


print (f'Shape of X: {X.shape}')
print (f'Shape of y: {y.shape}')



Shape of training data set: (162500, 166)
Shape of test data set: (160651, 164)
[-0.41764802 -0.01699412  0.01510229 ... -0.29044643 -0.21449289
 -1.1436536 ]
Shape of X: (121495, 160)
Shape of y: (121495,)


In [3]:
shap_variables = ['p_eCluster',
 'p_rawECluster',
 'p_eAccCluster',
 'p_ecore',
 'p_E7x11_Lr2',
 'p_E7x7_Lr2',
 'p_e277',
 'p_eClusterLr2']


cluster_variables = ['p_Rhad','p_Reta','p_deltaEta1', 'p_sigmad0',
	'p_Rphi',	
	'p_ambiguityType',	
	'p_ethad',
	'p_numberOfInnermostPixelHits']

norm_X_cluster = preprocessing.MinMaxScaler()
sc_X_cluster = preprocessing.StandardScaler()
X_cluster = train[cluster_variables]
X_cluster = sc_X_cluster.fit_transform(X_cluster)


kms_variables = ['p_E_Lr0_HiG', 'p_eClusterLr0', 'p_E5x7_Lr0', 'p_E7x11_Lr0', 'p_E7x7_Lr0', 'p_E3x5_Lr0', 'p_nCells_Lr0_HiG','p_deltaEta0']
norm_X_cluster = preprocessing.MinMaxScaler()
sc_X_kms = preprocessing.StandardScaler()
X_kms = train[kms_variables]
# X_kms = sc_X_kms.fit_transform(X_kms)

In [4]:
# parameters


default_base = {
    "quantile": 0.3,
    "eps": 0.3,
    "damping": 0.9,
    "preference": -200,
    "n_neighbors": 10,
    "n_clusters": 3,
    "min_samples": 20,
    "xi": 0.05,
    "min_cluster_size": 0.1,
}


In [5]:
# def gmm():
#     params = default_base.copy()
#     bandwidth = cluster.estimate_bandwidth(X, quantile=params["quantile"])
#     gaussianmm = mixture.GaussianMixture(
#         n_components=params["n_clusters"], covariance_type="full"
#     )
#     gaussianmm.fit(X)
    


In [6]:
# import time
# import warnings
# from tqdm.notebook import tqdm, trange    # Progress bar.

# import numpy as np
# import matplotlib.pyplot as plt

# from sklearn import cluster, datasets, mixture
# from sklearn.neighbors import kneighbors_graph
# from sklearn.preprocessing import StandardScaler
# from itertools import cycle, islice

# np.random.seed(0)

# # ============
# # Set up cluster parameters
# # ============
# plt.figure(figsize=(9 * 2 + 3, 13))
# plt.subplots_adjust(
#     left=0.02, right=0.98, bottom=0.001, top=0.95, wspace=0.05, hspace=0.01
# )

# plot_num = 1

# default_base = {
#     "quantile": 0.3,
#     "eps": 0.3,
#     "damping": 0.9,
#     "preference": -200,
#     "n_neighbors": 10,
#     "n_clusters": 3,
#     "min_samples": 20,
#     "xi": 0.05,
#     "min_cluster_size": 0.1,
# }





# # update parameters with dataset-specific values
# params = default_base.copy()

# # normalize dataset for easier parameter selection

# # estimate bandwidth for mean shift
# bandwidth = cluster.estimate_bandwidth(X_cluster, quantile=params["quantile"])

# # connectivity matrix for structured Ward
# connectivity = kneighbors_graph(
#     X_cluster, n_neighbors=params["n_neighbors"], include_self=False
# )
# # make connectivity symmetric
# connectivity = 0.5 * (connectivity + connectivity.T)

# # ============
# # Create cluster objects
# # ============
# ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
# two_means = cluster.MiniBatchKMeans(n_clusters=params["n_clusters"])
# ward = cluster.AgglomerativeClustering(
#     n_clusters=params["n_clusters"], linkage="ward", connectivity=connectivity
# )
# spectral = cluster.SpectralClustering(
#     n_clusters=params["n_clusters"],
#     eigen_solver="arpack",
#     affinity="nearest_neighbors",
# )
# dbscan = cluster.DBSCAN(eps=params["eps"])
# optics = cluster.OPTICS(
#     min_samples=params["min_samples"],
#     xi=params["xi"],
#     min_cluster_size=params["min_cluster_size"],
# )
# affinity_propagation = cluster.AffinityPropagation(
#     damping=params["damping"], preference=params["preference"],
# )
# average_linkage = cluster.AgglomerativeClustering(
#     linkage="average",
#     affinity="cityblock",
#     n_clusters=params["n_clusters"],
#     connectivity=connectivity,
# )
# birch = cluster.Birch(n_clusters=params["n_clusters"])
# gmm = mixture.GaussianMixture(
#     n_components=params["n_clusters"], covariance_type="full"
# )

# clustering_algorithms = (
#     ("MiniBatch\nKMeans", two_means),
#     ("Affinity\nPropagation", affinity_propagation),
#     ("MeanShift", ms),
#     ("Spectral\nClustering", spectral),
#     ("Ward", ward),
#     ("Agglomerative\nClustering", average_linkage),
#     ("DBSCAN", dbscan),
#     ("OPTICS", optics),
#     ("BIRCH", birch),
#     ("Gaussian\nMixture", gmm),
# )

# for name, algorithm in clustering_algorithms:
#     t0 = time.time()

#     # catch warnings related to kneighbors_graph
#     with warnings.catch_warnings():
#         warnings.filterwarnings(
#             "ignore",
#             message="the number of connected components of the "
#             + "connectivity matrix is [0-9]{1,2}"
#             + " > 1. Completing it to avoid stopping the tree early.",
#             category=UserWarning,
#         )
#         warnings.filterwarnings(
#             "ignore",
#             message="Graph is not fully connected, spectral embedding"
#             + " may not work as expected.",
#             category=UserWarning,
#         )
#         algorithm.fit(X_cluster)

#     t1 = time.time()
#     if hasattr(algorithm, "labels_"):
#         y_pred = algorithm.labels_.astype(int)
#     else:
#         y_pred = algorithm.predict(X_cluster)

#     plt.subplot(1, len(clustering_algorithms), plot_num)
    
#     plt.title(name, size=18)


#     colors = np.array(
#         list(
#             islice(
#                 cycle(
#                     [
#                         "#377eb8",
#                         "#ff7f00",
#                         "#4daf4a",
#                         "#f781bf",
#                         "#a65628",
#                         "#984ea3",
#                         "#999999",
#                         "#e41a1c",
#                         "#dede00",
#                     ]
#                 ),
#                 int(max(y_pred) + 1),
#             )
#         )
#     )
#     # add black color for outliers (if any)
#     colors = np.append(colors, ["#000000"])
#     plt.scatter(X_cluster[:, 0], X_cluster[:, 1], s=10, color=colors[y_pred])

#     plt.xlim(-2.5, 2.5)
#     plt.ylim(-2.5, 2.5)
#     plt.xticks(())
#     plt.yticks(())
#     plt.text(
#         0.99,
#         0.01,
#         ("%.2fs" % (t1 - t0)).lstrip("0"),
#         transform=plt.gca().transAxes,
#         size=15,
#         horizontalalignment="right",
#     )
#     plot_num += 1

# plt.show()

In [7]:
def k_mean(ncluster, X):
    k_means = cluster.MiniBatchKMeans(n_clusters=ncluster)
    k_means.fit(X)
    y_pred_kms = k_means.predict(X)

    return y_pred_kms

def gaussian(ncluster,X):
    gmm = mixture.GaussianMixture(
        n_components=ncluster)
    gmm.fit(X)
    y_pred_gmm = gmm.predict(X)

    return y_pred_gmm

def birch(ncluster, X):
    birch1 = cluster.Birch(n_clusters=ncluster)
    birch1.fit(X)
    y_pred_birch = birch1.predict(X_cluster)

    return y_pred_birch

# plt.title('Original data (3 groups)')
# plt.scatter(X_cluster.T[0], X_cluster.T[3], c=y_pred)
# plt.tight_layout()
# plt.show()

In [10]:
ncluster = 5
y_pred_kms = k_mean(ncluster, X_kms)
y_pred_gmm = gaussian(ncluster, X_kms)
y_pred_birch = birch(ncluster, X_kms)
for i in range(ncluster):
    print(f'kmean: {np.sum(y_pred_kms==i)}')
    print(f'gmm: {np.sum(y_pred_gmm==i)}')
    print(f'birch: {np.sum(y_pred_birch==i)}')

MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 4096 or by setting the environment variable OMP_NUM_THREADS=1


In [ ]:
# sum1 = 0
# n = 9
# # y_predict = k_mean(n)
# accuracy = np.zeros(n)
# sum1 = np.zeros(n)
# nexp = 100
# total = np.zeros([n,nexp])

# for m in range(nexp):
#     for i in range(2, n):
#         pred = np.zeros(n)
#         y_predict = k_mean(i)
#         for j in range(n):
#             pred[j] = np.sum(y_predict==j)
        
#         sum1[i] = np.max(pred)
    
#     total[:,m] = sum1


# accuracy = np.abs((121495 - np.mean(total, axis=1)) / 121495)

# # print(f'sum is :{sum1}')
# print(accuracy)

In [ ]:
import time
import warnings
from tqdm.notebook import tqdm, trange    # Progress bar.

import numpy as np
import matplotlib.pyplot as plt

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

params = {
"quantile": 0.3,
"eps": 0.3,
"damping": 0.9,
"preference": -200,
"n_neighbors": 2,
"n_clusters": 5,
"min_samples": 20,
"xi": 0.05,
"min_cluster_size": 0.1,
}
ncluster = 5


# estimate bandwidth for mean shift
# bandwidth = cluster.estimate_bandwidth(X_cluster, quantile=params["quantile"])

# # connectivity matrix for structured Ward
# connectivity = kneighbors_graph(
#     X_cluster, n_neighbors=params["n_neighbors"], include_self=False
# )
# # make connectivity symmetric
# connectivity = 0.5 * (connectivity + connectivity.T)

# ============
# Create cluster objects
# ============
# ms = cluster.MeanShift(bin_seeding=True)
# ms.fit(X_cluster)

two_means = cluster.MiniBatchKMeans(n_clusters=params["n_clusters"])
two_means.fit(X_cluster)
y_pred_kmean = two_means.predict(X_cluster)
for i in range(ncluster):
    print(f'kmean: {np.sum(y_pred_kmean==i)}')

# ward = cluster.AgglomerativeClustering(
#     n_clusters=params["n_clusters"], linkage="ward", connectivity=connectivity
# )
# ward.fit(X_cluster)

# spectral = cluster.SpectralClustering(
#     n_clusters=params["n_clusters"],
#     eigen_solver="arpack",
#     affinity="nearest_neighbors",
# )
# spectral.fit(X_cluster)

# dbscan = cluster.DBSCAN(algorithm='auto')
# dbscan.fit(X_cluster)

# optics = cluster.OPTICS(
#     min_samples=params["min_samples"],
#     xi=params["xi"],
#     min_cluster_size=params["min_cluster_size"],
# )
# optics.fit(X_cluster)

# affinity_propagation = cluster.AffinityPropagation(
#     damping=params["damping"], preference=params["preference"],
# )
# affinity_propagation.fit(X_cluster)

# average_linkage = cluster.AgglomerativeClustering(
#     linkage="average",
#     affinity="cityblock",
#     n_clusters=params["n_clusters"], connectivity=connectivity
# )
# average_linkage.fit(X_cluster)

# birch = cluster.Birch(n_clusters=params["n_clusters"], threshold=0.7)
# birch.fit(X_cluster)
# y_pred_birch = birch.predict(X_cluster)
# for i in range(ncluster):
#     print(f'birch: {np.sum(y_pred_birch==i)}')

def gaussian(ncluster,X):
    gmm = mixture.GaussianMixture(
        n_components=ncluster)
    gmm.fit(X)
    y_pred_gmm = gmm.predict(X)

    return y_pred_gmm
    for i in range(ncluster):
        print(f'gmm: {np.sum(y_pred_gmm==i)}')



# clustering_algorithms = (
#     ("MiniBatch\nKMeans", two_means),
#     ("Affinity\nPropagation", affinity_propagation),
#     ("MeanShift", ms),
#     ("Spectral\nClustering", spectral),
#     ("Ward", ward),
#     ("Agglomerative\nClustering", average_linkage),
#     ("DBSCAN", dbscan),
#     ("OPTICS", optics),
#     ("BIRCH", birch),
#     ("Gaussian\nMixture", gmm),
# )

# for name, algorithm in clustering_algorithms:

#     # catch warnings related to kneighbors_graph
#     with warnings.catch_warnings():
#         warnings.filterwarnings(
#             "ignore",
#             message="the number of connected components of the "
#             + "connectivity matrix is [0-9]{1,2}"
#             + " > 1. Completing it to avoid stopping the tree early.",
#             category=UserWarning,
#         )
#         warnings.filterwarnings(
#             "ignore",
#             message="Graph is not fully connected, spectral embedding"
#             + " may not work as expected.",
#             category=UserWarning,
#         )
#         algorithm.fit(X_cluster)

#     if hasattr(algorithm, "labels_"):
#         y_pred = algorithm.labels_.astype(int)
#     else:
#         y_pred = algorithm.predict(X_cluster)



MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 4096 or by setting the environment variable OMP_NUM_THREADS=1


kmean: 118956
kmean: 3729
kmean: 13914
kmean: 16749
kmean: 9152


In [ ]:
# from kmeans_interp.kmeans_feature_imp import KMeansInterp

# kms = KMeansInterp(
# 	n_clusters=5,
# 	ordered_feature_names=X.columns.tolist(), 
# 	feature_importance_method='wcss_min', # or 'unsup2sup'
# ).fit(X.values)

# kms.feature_importances_[0][:8]




